In [25]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import requests
import haversine as hs
import time


load_dotenv('config.env')
map_api_key = os.getenv('GOOGLE_MAP_API_KEY')
housing_csv = pd.read_csv('../../Downloads/NY-House-Dataset.csv')
housing_df = housing_csv.iloc[:10,:]
housing_df.head()


,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [66]:
class NearByStoreSearch():
    """
    A class to search the number of nearby stores using the Google Maps API.
    
    Attributes:
        keywords (list of str): A list of the store names to search for. 
        location (list of tuple): A list of tuples, where each tuple contains two float values of the latitude and longitude of the location from which the search is run.
        radius (int): the search radius in meters within which to find the stores specified the keywords, default is set to 500m, which is considered to be a walking distance.
        key (str): Google Maps API registered to use Places API.
        
    Output:
    """
    def __init__(self, keywords, locations, radius=500, key=map_api_key):
        self.keywords = keywords
        self.locations = locations
        self.radius = radius
        self.key = key
        self.base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    def num_of_stores_nearby(self):
        radius_in_kilo = self.radius / 1000 # convert meters to kilos
        result = {'location':[]}
        for k in self.keywords:
            result[k] = []

        for lat, lng in self.locations:
            result['location'].append((lat, lng))
            for keyword in self.keywords:
                more_pages = True
                next_page_token = None
                num_of_stores_nearby = 0

                while more_pages:
                    if next_page_token: 
                        params = f'keyword={keyword}&location={str(lat)}%2C{str(lng)}&radius={self.radius}&key={self.key}&pagetoken={next_page_token}'
                    else:
                        params = f'keyword={keyword}&location={str(lat)}%2C{str(lng)}&radius={self.radius}&key={self.key}'
                    
                    time.sleep(2)
                    response = requests.get(self.base_url + params)
                    if response.status_code == 200:
                        data = response.json()
                        results = data.get('results')
                        next_page_token = data.get('next_page_token')
                        results_locs = [(res['geometry']['location']['lat'], res['geometry']['location']['lng']) for res in results]
                        distance_to_results = [hs.haversine((lat, lng), loc) for loc in results_locs]
                        num_of_stores_nearby += sum([1 for d in distance_to_results if d < radius_in_kilo])

                        more_pages = bool(next_page_token)
                    else:
                        print(f'Failed to fetch results for {keyword} at {(lat, lng)}')
                        break
                result[keyword].append(num_of_stores_nearby)
                print(f'There are {num_of_stores_nearby} {keyword} within {self.radius}M from {(lat, lng)}')

        return result

    def num_of_good_restaurants(self):
        for lat, lng in self.locations:
            param = f'keyword=restaurants&location={str(lat)}%2C{str(lng)}&radius={self.radius}&key={self.key}'
            response 



In [67]:
keywords = ['Starbucks', 'Whole Foods', 'Trader Joes', 'McDonalds', 'Movie Theater', 'Equinox']
locations = list(zip(housing_df['LATITUDE'], housing_df['LONGITUDE']))
nearby_stores = NearByStoreSearch(keywords, locations)

In [69]:
housing_df_nearby_stores = nearby_stores.num_of_stores_nearby()

There are 10 Starbucks within 500M from (40.761255, -73.9744834)
There are 0 Whole Foods within 500M from (40.761255, -73.9744834)
There are 0 Trader Joes within 500M from (40.761255, -73.9744834)
There are 0 McDonalds within 500M from (40.761255, -73.9744834)
There are 8 Starbucks within 500M from (40.7663935, -73.9809909)
There are 1 Whole Foods within 500M from (40.7663935, -73.9809909)
There are 0 Trader Joes within 500M from (40.7663935, -73.9809909)
There are 1 McDonalds within 500M from (40.7663935, -73.9809909)
There are 0 Starbucks within 500M from (40.5418051, -74.1961086)
There are 0 Whole Foods within 500M from (40.5418051, -74.1961086)
There are 0 Trader Joes within 500M from (40.5418051, -74.1961086)
There are 0 McDonalds within 500M from (40.5418051, -74.1961086)
There are 10 Starbucks within 500M from (40.7613979, -73.9746128)
There are 0 Whole Foods within 500M from (40.7613979, -73.9746128)
There are 0 Trader Joes within 500M from (40.7613979, -73.9746128)
There are 0

In [70]:
housing_df_nearby_stores

{'location': [(40.761255, -73.9744834),
  (40.7663935, -73.9809909),
  (40.5418051, -74.1961086),
  (40.7613979, -73.9746128),
  (40.7672235, -73.9698561),
  (40.6743632, -73.9587248),
  (40.809448, -73.946777),
  (40.5950017, -74.1064235),
  (40.8215857, -73.8740892),
  (40.6157378, -73.9696944)],
 'Starbucks': [10, 8, 0, 10, 3, 1, 1, 0, 0, 0],
 'Whole Foods': [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 'Trader Joes': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'McDonalds': [0, 1, 0, 0, 0, 0, 1, 0, 1, 0]}

In [37]:
d['location']

[(10, 30), (40, 93)]

In [34]:
d.get('page')

In [35]:
d['page']

KeyError: 'page'

In [39]:
bool(1)

True